# 3 · The Search for Equilibrium

**Observational record associated with the book**  
*Hidden Structure in the Apparent Chaos of Prime Numbers — Computational experiments through the Euler mirror*  
© Alvaro Costa, 2025

This notebook is part of a **canonical sequence of computational records**.  
It does **not** introduce new hypotheses, conjectures, or interpretative models.

Its sole purpose is to **record** the behaviour of arithmetic structures under an  
explicit, deterministic, and reproducible observational regime.

The full conceptual discussion is presented in the book.  
This notebook documents **only** the corresponding experiment.

**Licence:** Creative Commons BY–NC–ND 4.0  
Reading, execution, and citation are permitted.  
Modification, adapted redistribution, or independent commercial use are not permitted.


---

## 1. The Paradox of the Growing Debt

In the previous chapter, we observed the *pulse* of the system, $\Delta_\pi(x)$.  
We saw that, after a brief initial phase of euphoria (positive values), the curve descends into a region of negative values — and this region appears  
to deepen as $x$ increases.

At first glance, this seems paradoxical.  
If the absolute difference between **Structuring** and **Stabilising** primes grows indefinitely, does the system not become increasingly **imbalanced**?  
Is the “debt” of the stabilising primes never repaid?

This impression, however, is only partial.  
The apparent imbalance is an **effect of scale**, not of structural breakdown.

---

## 2. The Correct Perspective · Relative Harmony

To understand systems that grow without bound, it is not sufficient to observe absolute differences — one must adopt a **relative perspective**.

Recall that the total number of primes is the sum of the two forces:

$
\pi(x) = \pi_S(x) + \pi_N(x)
$

Dividing both sides by (\pi(x)), we obtain a simple yet profound identity, which we call the **Law of Conservation of Primes**:

$
1 = \frac{\pi_S(x)}{\pi(x)} + \frac{\pi_N(x)}{\pi(x)}
$

This identity tells us that, although absolute counts may vary, the fractions of Structuring and Stabilising primes **always sum exactly to 1**.  
They are two aspects of a single dynamic equilibrium.

The **Prime Number Theorem** guarantees that, as (x \to \infty), both fractions converge to the same value: $\tfrac{1}{2}$.  
Structure and stability become, on average, indistinguishable.

Revisiting our measure of tension, $\Delta_\pi(x)$, we may therefore rewrite it in **normalised form**:

$
\frac{\Delta_\pi(x)}{\pi(x)} =
\frac{\pi_N(x)}{\pi(x)} - \frac{\pi_S(x)}{\pi(x)}
$

If each term tends to $\tfrac{1}{2}$, their difference tends to **zero**.  
The “debt” between the two forces may grow in absolute terms, but *relative to the whole*, it vanishes.  
Seen from afar, the system moves inexorably towards equilibrium.

---

## 3. Interactive Laboratory · Convergence to Equilibrium

The cell below creates a small experiment to visualise this harmonic convergence.
Increase the upper limit $N$ using the *slider* and observe the two plots.

**What to observe:**

1. **Plot 1:** The blue and orange curves — Structuring and Stabilising — approach the dashed line at $\tfrac{1}{2}$, as if attracted by a  
   field of symmetry.
3. **Plot 2:** The relative difference between the two forces oscillates strongly for small $x$, but these fluctuations are progressively  
    damped, and the curve converges smoothly towards equilibrium at $0$.


In [1]:
# Requirements: matplotlib, numpy, ipywidgets
# Run in Google Colab or Jupyter with the appropriate kernel.

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import time

# --- Colours (for visual consistency only) ---
BLUE   = "#0055A4"
ORANGE = "#FF7F11"

# --- Helper function: generate primes up to n ---
def primes_up_to(n: int) -> np.ndarray:
    """Generate an array containing all prime numbers up to n."""
    if n < 2:
        return np.array([], dtype=int)
    sieve = np.ones(n + 1, dtype=bool)
    sieve[:2] = False
    for i in range(2, int(np.sqrt(n)) + 1):
        if sieve[i]:
            sieve[i * i : n + 1 : i] = False
    return np.flatnonzero(sieve)

# --- Main interactive function (optimised) ---
def plot_interactive_relative_harmony(log_N_max=4):
    
    N_max = int(10 ** log_N_max)
    print(f"Generating data up to x = {N_max:,}...")
    start_time = time.time()

    # --- 1. Vectorised and optimised data generation ---
    primes = primes_up_to(N_max)
    
    # Efficient cumulative prime counting array π(x)
    pi_cumulative = np.zeros(N_max + 1, dtype=int)
    if primes.size > 0:
        pi_cumulative[primes] = 1
    pi_cumulative = np.cumsum(pi_cumulative)

    # Compute all quantities at once using NumPy array operations
    x_range = np.arange(1, N_max + 1)
    pi_x_vals = pi_cumulative[x_range]
    pi_S_vals = pi_cumulative[x_range // 2]      # π(floor(x/2))
    pi_N_vals = pi_x_vals - pi_S_vals             # π(x) − π(floor(x/2))
    delta_pi_vals = pi_N_vals - pi_S_vals         # π_N(x) − π_S(x)
    
    # Prepare fractions and relative tension, avoiding division by zero
    pi_x_safe = np.where(pi_x_vals == 0, np.nan, pi_x_vals)
    frac_S = pi_S_vals / pi_x_safe
    frac_N = pi_N_vals / pi_x_safe
    delta_pi_ratio = delta_pi_vals / pi_x_safe
    
    end_time = time.time()
    print(f"-> Data generated in {end_time - start_time:.2f} seconds.")

    # --- 2. Plot construction ---
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, axes = plt.subplots(2, 1, figsize=(12, 9), sharex=True)

    # --- Plot 1: Law of Conservation ---
    ax1 = axes[0]
    ax1.semilogx(
        x_range, frac_S, lw=1.5,
        label=r"Fraction of Structuring primes: $\pi_S(x)/\pi(x)$",
        color=BLUE
    )
    ax1.semilogx(
        x_range, frac_N, lw=1.5,
        label=r"Fraction of Stabilising primes: $\pi_N(x)/\pi(x)$",
        color=ORANGE
    )
    ax1.axhline(
        0.5, ls="--", color='black', lw=1.0, alpha=0.7,
        label=r"Equilibrium limit (1/2)"
    )
    ax1.set_ylim(0, 1.05)
    ax1.set_ylabel("Fraction of total primes")
    ax1.set_title(
        "1. Law of Conservation: Fractions converge to 1/2",
        fontsize=14
    )
    ax1.legend(loc='upper right')

    # --- Plot 2: Convergence of Relative Tension ---
    ax2 = axes[1]
    ax2.semilogx(
        x_range, delta_pi_ratio, linewidth=1.5,
        color='black',
        label=r"$\Delta_\pi(x)/\pi(x)$"
    )
    ax2.axhline(
        0, linestyle="--", color='black', linewidth=1.2,
        alpha=0.7,
        label="Perfect equilibrium (y = 0)"
    )
    ax2.set_ylabel("Relative difference")
    ax2.set_xlabel("x (logarithmic scale)")
    ax2.set_title(
        r"2. Relative Harmony: $\Delta_\pi(x)/\pi(x)$ converges to zero",
        fontsize=14
    )
    ax2.legend(loc='upper right')

    fig.tight_layout(pad=2.0)
    plt.show()

# --- Create interactive widget ---
interact(
    plot_interactive_relative_harmony,
    log_N_max=widgets.IntSlider(
        min=2, max=7, step=1, value=4,
        description='Show N up to 10^',
        continuous_update=False
    )
);


interactive(children=(IntSlider(value=4, continuous_update=False, description='Show N up to 10^', max=7, min=2…

### A Quantum System in Disguise?

The oscillations of $\frac{\Delta_\pi(x)}{\pi(x)}$ resemble the behaviour of a quantum system relaxing towards equilibrium.  
The two forces — **Structuring** and **Stabilising** — behave like states of a two-level system, exchanging dominance until  
symmetry is restored.

The logarithmic scale plays a role analogous to an internal time, in which the initial chaos gradually gives way to  
spectral order.

Viewed through this lens, the number line is not static — it is an oscillatory field whose final harmony emerges precisely  
from the conflict that generated it.

---

> What appeared to be an infinite mismatch ultimately reveals itself as a harmony that grows with the scale itself.  
> The initial chaos is not an error — it is the cost of expansion.

---

**Methodological note:**  
The analogy with quantum systems is strictly descriptive.  
No physical structure, quantum operator, or dynamical model is assumed.  
The term is used solely to characterise the observable pattern of relaxation towards relative equilibrium.

This relative equilibrium does not eliminate the signal — it prepares it.  
In the next records, the same behaviour will be examined under spectral transformations, where the notion of harmony will cease  
to be merely proportional and will become structural.
